In [64]:
# from openmm import LangevinMiddleIntegrator

# LangevinMiddleIntegrator(300 * unit.kelvin, )

In [66]:
import openmm
from openmm import unit

from openmmtools import integrators

import mdtraj as md

import copy
import pickle
from openeye import oechem
import tqdm

In [67]:
# Load ALA->THR dipeptide rest htf
with open("atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

# # Load RBD:ACE2 rest htf
# with open("/data/chodera/zhangi/perses_benchmark/neq/15/202/202_complex.pickle", "rb") as f:
#     htf = pickle.load(f)

In [69]:
# Retrieve old positions and system 
# (in PointMutationExecutor, we will want to feed the gently equilibrated positions in before geometry engine)
positions = htf.old_positions(htf.hybrid_positions)
system = htf._topology_proposal.old_system
topology = htf._topology_proposal.old_topology

# Stage 1: 10 K steps minimization, restrain protein heavy atoms (force constant = 100 kcal/molA^2)

In [70]:
# Add restraint
system_copy = copy.deepcopy(system)

# force = openmm.CustomExternalForce("k*((x-x0)^2+(y-y0)^2+(z-z0)^2)")
# force.addGlobalParameter("k", 100.0*unit.kilocalories_per_mole/unit.angstroms**2)
# force.addPerParticleParameter("x0")
# force.addPerParticleParameter("y0")
# force.addPerParticleParameter("z0")

traj = md.Trajectory(positions, md.Topology.from_openmm(topology))
selection_indices = traj.topology.select('protein and not type H')

# for index in selection_indices:
#     force.addParticle(index, positions[index].value_in_unit(unit.nanometers))

# system_copy.addForce(force)

custom_cv_force = openmm.CustomCVForce('(K_RMSD/2)*(RMSD)^2')
custom_cv_force.addGlobalParameter('K_RMSD', 100.0*unit.kilocalories_per_mole/unit.angstroms**2)
rmsd_force = openmm.RMSDForce(positions, selection_indices)
custom_cv_force.addCollectiveVariable('RMSD', rmsd_force)
system_copy.addForce(custom_cv_force)


5

In [71]:
# Set up integrator
temperature = 300 * unit.kelvin
collision_rate = 1.0 / unit.picoseconds
timestep = 4.0 * unit.femtoseconds
splitting = 'V R O R V'
constraint_tolerance = 1e-08
integrator = integrators.LangevinIntegrator(temperature=temperature, 
                                            collision_rate=collision_rate, 
                                            timestep=timestep, 
                                            splitting=splitting, 
                                            constraint_tolerance=constraint_tolerance)

# Set up context
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system_copy, integrator, platform)
context.setPeriodicBoxVectors(*system_copy.getDefaultPeriodicBoxVectors())
context.setPositions(positions)
context.setVelocitiesToTemperature(temperature)

# Minimize
openmm.LocalEnergyMinimizer.minimize(context, maxIterations=10000)

In [72]:
positions = context.getState(getPositions=True).getPositions(asNumpy=True)

# Stage 2: 100 K steps MD, T = 100/300 K, NVT, restrain protein heavy atoms (force constant = 100 kcal/molA^2), bath coupling constant = 0.1 ps

Questions: What does "100/300" means? To make this NVT, do I just remove the barostat? What is a bath coupling constant? Why is this nan-ing?

In [73]:
# Add restraint
system_copy = copy.deepcopy(system)

# force = openmm.CustomExternalForce("k*((x-x0)^2+(y-y0)^2+(z-z0)^2)")
# force.addGlobalParameter("k", 100.0*unit.kilocalories_per_mole/unit.angstroms**2)
# force.addPerParticleParameter("x0")
# force.addPerParticleParameter("y0")
# force.addPerParticleParameter("z0")

traj = md.Trajectory(positions, md.Topology.from_openmm(topology))
selection_indices = traj.topology.select('protein and not type H')

# for index in selection_indices:
#     force.addParticle(index, positions[index].value_in_unit(unit.nanometers))

# system_copy.addForce(force)

custom_cv_force = openmm.CustomCVForce('(K_RMSD/2)*(RMSD)^2')
custom_cv_force.addGlobalParameter('K_RMSD', 100.0*unit.kilocalories_per_mole/unit.angstroms**2)
rmsd_force = openmm.RMSDForce(positions, selection_indices)
custom_cv_force.addCollectiveVariable('RMSD', rmsd_force)
system_copy.addForce(custom_cv_force)


5

In [75]:
# Set up integrator
temperature = 300 * unit.kelvin
collision_rate = 1.0 / unit.picoseconds
timestep = 4.0 * unit.femtoseconds
splitting = 'V R O R V'
constraint_tolerance = 1e-08
integrator = integrators.LangevinIntegrator(temperature=temperature, 
                                            collision_rate=collision_rate, 
                                            timestep=timestep, 
                                            splitting=splitting, 
                                            constraint_tolerance=constraint_tolerance)

# Set up context
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system_copy, integrator, platform)
context.setPeriodicBoxVectors(*system_copy.getDefaultPeriodicBoxVectors())
context.setPositions(positions)
context.setVelocitiesToTemperature(temperature)

# Run MD
for _ in tqdm.tqdm_notebook(range(100000)):
    integrator.step(1)

/scratch/lsftmp/4773884.tmpdir/ipykernel_756539/4156506540.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm.tqdm_notebook(range(100000)):


  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
positions = context.getState(getPositions=True).getPositions(asNumpy=True)

# Stage 3: 100 K steps MD, T = 300 K, NPT, restrain protein heavy atoms (force constant = 100 kcal/molA^2), bath coupling constant = 0.1 ps

# Stage 4: 250 K steps MD, T = 300 K, NPT,  restrain protein heavy atoms (force constant = 10 kcal/molA^2), bath coupling constant = 0.5 ps

# Stage 5: 10 K steps minimization, restrain protein backbone atoms (CA, C, N) (force constant = 10 kcal/molA^2)

# Stage 6: 100 K steps MD, T = 300 K, NPT, restraint protein backbone atoms (CA, C, N) (force constant = 10 kcal/molA^2), bath coupling constant = 0.5 ps

# Stage 7: 100 K steps MD, restrain protein backbone atoms (CA, C, N) (force constant = 1 kcal/molA^2), bath coupling constant = 0.5 ps

# Stage 8: 100 K steps MD, restrain protein backbone atoms (CA, C, N) (force constant = 0.1 kcal/molA^2), bath coupling constant = 0.5 ps

# Stage 9: 2500 K steps MD, 300 K, NPT, bath coupling constant = 0.5 ps